In [1]:
# Allow python to import modules in our src folders
import sys
sys.path.append("src/")

from preprocessing.load_dataset import *

## Importing the CMU dataset

We load the 4 `.tsv` and `.txt` files of the CMU dataset into 4 different dataframes

In [2]:
characters_df = load_characters_df()
movies_df = load_movies_df()
plot_df = load_plot_df()
tvtropes_tf = load_tvtropes_df()

In [3]:
movies_df.sample(5)

,wiki_movie_id,freebase_movie_id,movie_name,movie_release_date,box_office,movie_runtime,movie_languages,movie_countries,movie_genres
4105,12892728,/m/02xb77x,Ethel's Romeos,1915,NaN,NaN,{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/02hmvc': 'Short Film', '/m/06ppq': 'Silen..."
9527,10484958,/m/02qfjv6,Don's Fountain of Youth,NaN,NaN,NaN,{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/02hmvc': 'Short Film', '/m/05p553': 'Come..."
36487,30971426,/m/0gg6jkv,Last Resort,1986,NaN,80.0,{},{'/m/09c7w0': 'United States of America'},"{'/m/01z4y': 'Comedy', '/m/09q17': 'Slapstick'}"
48764,12073859,/m/02vnycn,Porky's Pet,1936,NaN,NaN,{'/m/02h40lc': 'English Language'},{'/m/09c7w0': 'United States of America'},"{'/m/02hmvc': 'Short Film', '/m/01z4y': 'Comed..."
9812,10732804,/m/02qnl2y,The Glamorous Life of Sachiko Hanai,2003-10-14,NaN,90.0,"{'/m/03_9r': 'Japanese Language', '/m/02h40lc'...",{'/m/03_3d': 'Japan'},"{'/m/01jfsb': 'Thriller', '/m/0gsy3b': 'Sex co..."


In [4]:
print(f"Number of movies loaded: {len(movies_df)}")

Number of movies loaded: 81741


In [6]:
plot_df.sample(5)

,wiki_movie_id,plot_summary
33017,2750041,A backwards sequence is shown. It starts with ...
21252,35587809,"Calvin Weir-Fields is a young novelist, strug..."
27111,12737417,The family of Justice Saxena is extremely happ...
13398,9001270,The film is set in post-independence India whe...
34600,18456072,"Broadway, New Year's Eve, 1928. A muckraking r..."


In [7]:
tvtropes_tf.sample(5)

,trope_name,character_data
402,retired_outlaw,"{'char': 'Butch Cassidy', 'movie': 'Butch Cass..."
47,broken_bird,"{'char': 'Chelsea Deardon', 'movie': 'Legal Ea..."
129,corrupt_corporate_executive,"{'char': 'Ratchet', 'movie': 'Robots', 'id': '..."
328,junkie_prophet,"{'char': 'Agatha', 'movie': 'Minority Report',..."
33,bounty_hunter,"{'char': 'Toombs', 'movie': 'The Chronicles of..."


In [8]:
# extract years from the release date of movies
movies_df.loc[:, 'movie_release_year'] = movies_df['movie_release_date'].astype(str).str[:4]
movies_df = movies_df[movies_df['movie_release_year'].str.contains("nan")==False]
movies_df['movie_release_year'] = movies_df['movie_release_year'].astype(int)


In [9]:
wrong_dates = movies_df[movies_df['movie_release_year'] < 1800].index
# there is only one movie that has the release date 1010, so we can change 1010 to 2010
movies_df.loc[wrong_dates, 'movie_release_year']= 2010
movies_df.loc[wrong_dates, 'movie_release_date'] ='2010-12-02'

## Perfomance of the movie in terms of revenue

In [9]:
# how many missing values in the column box_office
percentage_missing = movies_df['box_office'].isna().sum() / len(movies_df) * 100
print("{:.2f}% values for the box office revenue are missing.".format(percentage_missing))

88.87% values for the box office revenue are missing.


## Extract ratings from the IMDB rating dataset

file: title.rating.tsv
-    tconst: unique identifier of the movie
-    averageRating: average of user ratings
-    numVotes: number of ratings submitted for the movie


In [16]:
imdb_ratings_df = load_imdb_ratings()
imdb_title_df = load_imdb_title_basics()
imdb_principals_df = load_imdb_title_principals()
imdb_people_df = load_imdb_person_basics()

In [17]:
print(movies_df.count())
print('==========')
print(imdb_title_df.count())

wiki_movie_id         74839
freebase_movie_id     74839
movie_name            74839
movie_release_date    74839
box_office             8328
movie_runtime         58631
movie_languages       74839
movie_countries       74839
movie_genres          74839
movie_release_year    74839
dtype: int64
tconst            661815
titleType         661815
primaryTitle      661815
originalTitle     661815
isAdult           661815
startYear         661815
endYear           661815
runtimeMinutes    661815
genres            661815
dtype: int64


In [18]:
# Extract the movies from the title.basics imbd dataset
imdb_title_df.sample(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
3214639,tt14110690,movie,Vanakkam Da Mappilei,Vanakkam Da Mappilei,0,2021,\N,162,Comedy
252296,tt0263501,movie,Jurgais pari,Jurgais pari,0,1944,\N,69,"Drama,History,Musical"
6836970,tt28433071,movie,Princess's Romance,Princess's Romance,0,2023,\N,\N,Action
4283470,tt1614992,movie,I Want So Much to Live,I Want So Much to Live,0,2009,\N,83,"Documentary,Drama"
6442684,tt2710826,movie,AfterDeath,AfterDeath,0,2015,\N,88,"Horror,Mystery,Thriller"


In [19]:
imdb_people_df.sample(5)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
5565840,nm14995081,Tau Braun,\N,\N,NaN,tt22696252
252954,nm0267003,Tim Faraday,\N,\N,"actor,director,producer","tt0808096,tt3569230,tt0208092,tt1139328"
1061000,nm10150070,Ivan Aguirre,\N,\N,actor,tt7933566
6520076,nm2185071,Patrick Queen,\N,\N,"miscellaneous,editorial_department,camera_depa...","tt9814218,tt3097248,tt6223802,tt5976986"
12805530,nm9792012,Skrotes,\N,\N,composer,tt8321498


In [20]:
imdb_principals_df.sample(5)

,tconst,ordering,nconst,category,job,characters
24420608,tt15564846,7,nm12374146,writer,\N,\N
38145263,tt27609766,2,nm0110828,writer,head writer,\N
18788392,tt13648152,2,nm6209025,self,\N,"[""Self - Co-Host""]"
13270950,tt11703382,5,nm1168198,director,\N,\N
5412367,tt0725520,3,nm0429809,self,\N,"[""Self""]"


In [21]:
# merge with ratings
imdb_ratings_meta = imdb_ratings_df.merge(imdb_title_df, on='tconst', how='inner')
imdb_ratings_meta.head()

,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000009,5.3,207,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
1,tt0000147,5.3,484,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,100,"Documentary,News,Sport"
2,tt0000502,4.1,15,movie,Bohemios,Bohemios,0,1905,\N,100,\N
3,tt0000574,6.0,854,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"
4,tt0000591,5.0,21,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama


## Merge the CMU dataset with the IMDB

We first import the translation dataframe, which includes both the IMDB id and the Freebase id, enabling us to combine the IMDB dataset with the CMU dataset.

In [15]:
translation_id = load_translation_df()
translation_id.columns

Index(['imdb_id', 'freebase_id'], dtype='object')

Then we merge the IMDB ratings with the CMU dataset

In [16]:
imdb_ratings_translated_df = imdb_ratings_df.merge(translation_id, how='inner', left_on='tconst', right_on='imdb_id')
merged_df = imdb_ratings_translated_df.merge(movies_df, how='inner', left_on='freebase_id', right_on='freebase_movie_id')

In [17]:
print(f"The merged dataframe contains {len(merged_df)} movies")

The merged dataframe contains 52003 movies


## Load personas

In [10]:
personas_df = load_personas()

In [11]:
personas_df

,freebase_id,movie_name,secondary_name,full_name,token_occurences,estimated_trope,trope_distrib
0,e7,The Trap,she,her,5,1,"[0.0, 0.63265, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
1,e8,The Trap,she,her,5,1,"[0.0, 0.77551, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
2,e12,The Trap,he,his,7,19,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,e19,The Trap,she,her,6,1,"[0.0, 0.67347, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
4,e4,The Trap,Eve,Eve,11,23,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...
178729,e11,Una Aventura Llamada Menudo,Concepci,Danny Concepci,3,1,"[0.0, 0.22449, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
178730,e10,Una Aventura Llamada Menudo,he,the son of C sar Concepci n Jr.,4,44,"[0.0, 0.02041, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
178731,e1,Una Aventura Llamada Menudo,they,their,6,10,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
178732,e0,Plaza de Almas,he,his,6,1,"[0.0, 0.46939, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."


In [22]:
personas_df.sample(5)

,freebase_id,movie_name,secondary_name,full_name,token_occurences,estimated_trope,trope_distrib
105638,e6,The Four-Faced Liar,girlfriend,a girlfriend because no one can hold her inter...,5,26,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
178427,e5,The War Between Men and Women,she,her,8,26,"[0.0, 0.34694, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
166965,/m/0bvsnb5,Stomp the Yard,Jackie,Jackie,6,26,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
137223,e23,Kohtalon kirja,friend,his friend who was on the line of fire,3,9,"[0.0, 0.02041, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
8902,e11,That's My Mommy,Jerry,Jerry,17,24,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
